# TASK:

В .csv файле представлена совокупность данных:
 - заголовок новости,
 - тема новости,
 - текст новости.

Все слова в тексте лемматизированы, а сами тексты очищены от стоп-слов.

Ссылка на .csv файл

1. Вам необходимо выбрать и сравнить между собой не менее трех различных алгоритмов **классификации текстов**,

2. проанализировать полученные результаты,

3. сделать обоснованные выводы о применимости выбранных методов и качестве решения задачи.

В качестве ответа необходимо загрузить .ipynb файл с кодом, комментариями и выводами.

Успехов!

# Libs import

In [2]:
import numpy as np
import pandas as pd
import random

random.seed(42)

In [30]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU, LSTM, Activation, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import matplotlib.pyplot as plt
%matplotlib inline

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('drive/MyDrive/articles_lemmatized_no_SW.csv', index_col=0)
df = df.reset_index().drop('index', axis = 1)
df.head()

,headline,body,topic
0,Теперь подарки можно выбрать на сайте при помо...,поиск подарок любой праздник гораздо просто бл...,IT
1,Тренды розничной торговли 2016,платформа соцсеть продолжать служить шопинг-пл...,IT
2,Порталы сравнения цен вынуждают мелких ритейле...,последний время активно расти конкуренция обла...,IT
3,9 беспроигрышных подарков для мужчины,каждый накануне февраль возникать сложный вопр...,IT
4,Китайские производители сокращают свой бизнес ...,прекращать поставка новый автомобиль дилер рос...,Авто


# Data first view

- Количество классов 37 (важно: выборка крайне не сбалансирована, экземпляров одного класса +- 26%, другого 0.0001%) => Некоторые классы детектироваться не будут
- Описание данных (мин/средние/макс) кол-во слов/символов в текстах

In [ ]:
print(df.iloc[0].headline, '\n')
print(df.iloc[0].body, '\n')
print(df.iloc[0].topic, '\n')

Теперь подарки можно выбрать на сайте при помощи заданных критериев 

поиск подарок любой праздник гораздо просто благодаря использование параметр поиск портал иметь функционал давать возможность выбирать подарок использовать детальный параметр поиск любой подарок второй половинка член семья друг родитель выбирать очень просто хороший подарок выбирать заблаговременно учитывать вкус пристрастие интерес человек адресовать выбирать подарок нужно учитывать кто вручать например косметика принимать дарить человек находиться близкий отношение точно постельный белье принимать дарить родственник пара правило этикет принимать считаться взгляд суеверие человек поэтому подарок платок слеза часы разлука суеверный человек дарить стоять стоимость подарок немаловажный фактор многое зависеть отношение человек самый дело важный сколько стоять главное приятный демонстрировать отношение человек поиск критерий помогать находить идеальный подарок начало нужно выбирать раздел подарок далее увидеть раздел сле

In [ ]:
df.dtypes

headline    object
body        object
topic       object
dtype: object

In [ ]:
df.describe()

,headline,body,topic
count,56500,56499,56500
unique,56396,56498,37
top,Спортивные выходные с «Известиями»,футбольный власть расследовать поведение росси...,Мир
freq,32,2,15131


In [ ]:

print("Классов :", len(df.topic.unique()), '\n')
counts = df.topic.value_counts()
percs = df.topic.value_counts().apply(lambda x: round(x/56500,6))
print(pd.concat([counts,percs],axis=1))

Классов : 37 

                                     topic     topic
Мир                                  15131  0.267805
Общество                             12147  0.214991
Политика                              7359  0.130248
Страна                                5280  0.093451
Экономика                             5032  0.089062
Спорт                                 4881  0.086389
Культура                              2635  0.046637
Армия                                 1295  0.022920
Наука                                  824  0.014584
Гаджеты & Телеком                      583  0.010319
РИО-2016                               449  0.007947
Авто                                   131  0.002319
Москва                                 118  0.002088
День в истории                         108  0.001912
Выборы в США                            96  0.001699
Выборы-2016                             92  0.001628
ПМЭФ                                    63  0.001115
День Победы                    

In [ ]:
body_words = df.body.str.split().str.len()
print(f"BODY WORDS min:{body_words.min()}; mean:{(body_words.mean())}; max:{body_words.max()}\n",'-'*40)

body_symb = df.body.str.len()
print(f"BODY SYMB min:{body_symb.min()}; mean:{(body_symb.mean())}; max:{body_symb.max()}\n",'-'*40)

head_words = df.headline.str.split().str.len()
print(f"HEAD WORDS min:{head_words.min()}; mean:{(head_words.mean())}; max:{head_words.max()}\n",'-'*40)

head_symb = df.headline.str.len()
print(f"HEAD SYMB min:{head_symb.min()}; mean:{(head_symb.mean())}; max:{head_symb.max()}\n",'-'*40)

BODY WORDS min:2.0; mean:168.30343899892034; max:5954.0
 ----------------------------------------
BODY SYMB min:9.0; mean:1484.4217596771625; max:48095.0
 ----------------------------------------
HEAD WORDS min:1; mean:7.913911504424779; max:15
 ----------------------------------------
HEAD SYMB min:4; mean:56.1170796460177; max:75
 ----------------------------------------


# Data Preparation

В этом разделе я описываю обработку данных:

- принято решение обучать модель только на текстах (НЕ включая заголовки), также произведена образка текстов до 100 слов СНАЧАЛА, тк новостные топики пишутся в стиле снижения важности информации

- удаление пустых занчений
- кодирование классов
- разбиение на тренировочную и тестовую выборки
- вкторизация текстов + фиксирование их длинны

In [74]:
print(df.shape)
df = df.dropna(axis=0)
print(df.shape)
num_words = 10000
max_len = 100


(56499, 3)
(56499, 3)


In [57]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df.topic.unique())
le.classes_
# le.transform([1, 1, 2, 6])
# le.inverse_transform([0, 0, 1, 2])

array(['IT', 'Авто', 'Армия', 'Выборы в США', 'Выборы-2016',
       'Гаджеты & Телеком', 'Деловая среда', 'День Победы',
       'День в истории', 'Знай наших', 'Известия: подробности',
       'Инновации', 'КНАУФ', 'Культура', 'Мир', 'Москва', 'Наука',
       'Недвижимость', 'Нефть и газ', 'Новости компаний', 'Образование',
       'Общество', 'ПМЭФ', 'Политика', 'Промышленность',
       'Путешествуем по России', 'РИО-2016', 'Россия - регионы', 'Спорт',
       'Страна', 'Технологическое предпринимательство', 'Транспорт',
       'Финансы', 'Формула лидерства', 'ЧЕ 2016', 'Шопинг', 'Экономика'],
      dtype=object)

In [58]:
labels = pd.DataFrame(le.transform(df.topic),columns=['class'], dtype=int)
df1 = pd.concat([df,labels], axis=1, join='inner').drop('topic', axis=1)
df1.head()


,headline,body,class
0,Теперь подарки можно выбрать на сайте при помо...,поиск подарок любой праздник гораздо просто бл...,0
1,Тренды розничной торговли 2016,платформа соцсеть продолжать служить шопинг-пл...,0
2,Порталы сравнения цен вынуждают мелких ритейле...,последний время активно расти конкуренция обла...,0
3,9 беспроигрышных подарков для мужчины,каждый накануне февраль возникать сложный вопр...,0
4,Китайские производители сокращают свой бизнес ...,прекращать поставка новый автомобиль дилер рос...,1


In [62]:
from sklearn.model_selection import train_test_split

# разбиение данных на тренировочную и тестовую части
df1 = df1.sample(frac=1).reset_index(drop=True)
x_train, x_test, y_train, y_test = train_test_split(df1.body, df1['class'], test_size=0.2, random_state=42)

In [63]:
# fit tokeniser
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(x_train)

#train
sequences = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(sequences, maxlen=max_len)

# test
# print(x_test[:,0])
sequences = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(sequences, maxlen=max_len)
# print(sequences[0])

# LSTM

В этом разделе приведена реализация простой RNN с ячейками LSTM.

Результаты предсказаний даже на такой простой сети сравнительно классные (на мой взгляд - самое перспективное мое решение).

In [88]:
lstm = Sequential()
lstm.add(Embedding(num_words, 64, input_length=max_len))
lstm.add(LSTM(128) )
lstm.add(Dense(37, activation='softmax'))


lstm.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
lstm.summary()


Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_28 (Embedding)    (None, 100, 64)           640000    
                                                                 
 lstm_6 (LSTM)               (None, 128)               98816     
                                                                 
 dense_36 (Dense)            (None, 37)                4773      
                                                                 
Total params: 743589 (2.84 MB)
Trainable params: 743589 (2.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [89]:
model_save_path = 'best_model_lstm.h5'
checkpoint_callback_lstm = ModelCheckpoint(model_save_path,
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

history = lstm.fit(x_train,
                    y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.1,
                    callbacks=[checkpoint_callback_lstm])

Epoch 1/5
318/318 [==============================] - ETA: 0s - loss: 1.8156 - accuracy: 0.4018
Epoch 1: val_accuracy improved from -inf to 0.59049, saving model to best_model_lstm.h5
318/318 [==============================] - 82s 248ms/step - loss: 1.8156 - accuracy: 0.4018 - val_loss: 1.3394 - val_accuracy: 0.5905
Epoch 2/5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


318/318 [==============================] - ETA: 0s - loss: 1.1044 - accuracy: 0.6532
Epoch 2: val_accuracy improved from 0.59049 to 0.68097, saving model to best_model_lstm.h5
318/318 [==============================] - 26s 82ms/step - loss: 1.1044 - accuracy: 0.6532 - val_loss: 1.0421 - val_accuracy: 0.6810
Epoch 3/5
  1/318 [..............................] - ETA: 4s - loss: 0.9266 - accuracy: 0.7109

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


318/318 [==============================] - ETA: 0s - loss: 0.8729 - accuracy: 0.7294
Epoch 3: val_accuracy improved from 0.68097 to 0.71571, saving model to best_model_lstm.h5
318/318 [==============================] - 16s 51ms/step - loss: 0.8729 - accuracy: 0.7294 - val_loss: 0.9188 - val_accuracy: 0.7157


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 4/5
318/318 [==============================] - ETA: 0s - loss: 0.7262 - accuracy: 0.7779
Epoch 4: val_accuracy improved from 0.71571 to 0.73761, saving model to best_model_lstm.h5
318/318 [==============================] - 18s 56ms/step - loss: 0.7262 - accuracy: 0.7779 - val_loss: 0.8520 - val_accuracy: 0.7376
Epoch 5/5
 13/318 [>.............................] - ETA: 2s - loss: 0.6377 - accuracy: 0.8191

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


318/318 [==============================] - ETA: 0s - loss: 0.6157 - accuracy: 0.8084
Epoch 5: val_accuracy improved from 0.73761 to 0.73783, saving model to best_model_lstm.h5
318/318 [==============================] - 11s 35ms/step - loss: 0.6157 - accuracy: 0.8084 - val_loss: 0.8505 - val_accuracy: 0.7378


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [90]:
lstm.evaluate(x=x_test,y=y_test)

354/354 [==============================] - 3s 8ms/step - loss: 0.8364 - accuracy: 0.7385


[0.836440920829773, 0.7384955883026123]

# CNN

В этой главе я пробую обучить сверточную нейросеть, использую таке же параметры компиляции.


In [102]:
cnn = Sequential()
cnn.add(Embedding(input_dim=100, output_dim=64, input_length=100))
cnn.add(Dropout(rate=0.5))
cnn.add(Dense(512))
cnn.add(Dropout(rate=0.5))
cnn.add(Conv1D(filters=250, kernel_size=3, activation='relu'))
cnn.add(GlobalMaxPool1D())
cnn.add(Dense(512))
cnn.add(Dropout(rate=0.5))
cnn.add(Dense(128))
cnn.add(Dropout(rate=0.5))
cnn.add(Dense(37, activation='softmax'))

cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
cnn.summary()

callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]


Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_32 (Embedding)    (None, 100, 64)           6400      
                                                                 
 dropout_45 (Dropout)        (None, 100, 64)           0         
                                                                 
 dense_57 (Dense)            (None, 100, 512)          33280     
                                                                 
 dropout_46 (Dropout)        (None, 100, 512)          0         
                                                                 
 conv1d_26 (Conv1D)          (None, 98, 250)           384250    
                                                                 
 global_max_pooling1d_23 (G  (None, 250)               0         
 lobalMaxPooling1D)                                              
                                                     

In [79]:
x_train[0]

array([ 183,  164, 2253,  137, 5894,   60,  541,  142, 7571,  831, 2915,
       1597,   90,  813,  349,  669,  366,  155, 3751,  837,  556,   27,
        463,  831,  158,  130, 4746, 5985,  831, 2050,  837,  106,   72,
       2696,  229, 9736, 1524,   49,   23,  836,  327,  304, 2604,  905,
        255,  702,  105,  255,  837,   56, 2783,  304, 2854, 1082,  106,
         99,   58,  278,   15,   25,    7,  541,  142,  326,  312,  124,
       1652,  342,   37,  653,  257,  237,  800,  202,  279,  335,   68,
          6,   75,  283,  166,  439,  110, 1510,  166,  720, 4497,   56,
        908,  202, 7488, 2248,  377,  908, 4237,  728,  166,  564,  183,
        113], dtype=int32)

In [103]:
history = cnn.fit(x_train, y_train,
                    epochs=10,
                    batch_size=64,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/10
636/636 [==============================] - 104s 159ms/step - loss: 1.7212 - accuracy: 0.4355 - val_loss: 1.4393 - val_accuracy: 0.5462 - lr: 0.0010
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


636/636 [==============================] - 37s 58ms/step - loss: 1.4508 - accuracy: 0.5266 - val_loss: 1.3850 - val_accuracy: 0.5500 - lr: 0.0010
Epoch 3/10
  6/636 [..............................] - ETA: 6s - loss: 1.3329 - accuracy: 0.5573

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


636/636 [==============================] - 28s 44ms/step - loss: 1.4089 - accuracy: 0.5399 - val_loss: 1.3383 - val_accuracy: 0.5591 - lr: 0.0010
Epoch 4/10
  9/636 [..............................] - ETA: 8s - loss: 1.4102 - accuracy: 0.5625

...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


636/636 [==============================] - 14s 23ms/step - loss: 1.3659 - accuracy: 0.5504 - val_loss: 1.3117 - val_accuracy: 0.5569 - lr: 0.0010
Epoch 8/10
636/636 [==============================] - 13s 21ms/step - loss: 1.3540 - accuracy: 0.5546 - val_loss: 1.2959 - val_accuracy: 0.5763 - lr: 0.0010
Epoch 9/10
 13/636 [..............................] - ETA: 5s - loss: 1.3909 - accuracy: 0.5385

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


636/636 [==============================] - 9s 15ms/step - loss: 1.3561 - accuracy: 0.5495 - val_loss: 1.2963 - val_accuracy: 0.5719 - lr: 0.0010
Epoch 10/10
636/636 [==============================] - 10s 16ms/step - loss: 1.3498 - accuracy: 0.5534 - val_loss: 1.2889 - val_accuracy: 0.5715 - lr: 0.0010


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [104]:
cnn.evaluate(x=x_test,y=y_test)

354/354 [==============================] - 2s 6ms/step - loss: 1.2808 - accuracy: 0.5718


[1.2808283567428589, 0.5717698931694031]

# MLP

Построение многослойной сети персептронов.

Результаты получились не очень(

In [98]:
mlp = Sequential()


mlp.add(Dense(512, input_shape=(100,)))
mlp.add(Activation('tanh'))
mlp.add(Dropout(0.5))
mlp.add(Dense(512, input_shape=(100,)))
mlp.add(Activation('relu'))
mlp.add(Dense(512, input_shape=(100,)))
mlp.add(Activation('relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(37))
mlp.add(Activation('softmax'))


mlp.summary()

# Add optimization method, loss function, and optimization value
mlp.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


mlp.fit(x_train, y_train, batch_size=64, epochs=10)

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 512)               51712     
                                                                 
 activation_15 (Activation)  (None, 512)               0         
                                                                 
 dropout_36 (Dropout)        (None, 512)               0         
                                                                 
 dense_47 (Dense)            (None, 512)               262656    
                                                                 
 activation_16 (Activation)  (None, 512)               0         
                                                                 
 dense_48 (Dense)            (None, 512)               262656    
                                                                 
 activation_17 (Activation)  (None, 512)             

In [99]:
mlp.evaluate(x_test, y_test, batch_size=64)

177/177 [==============================] - 1s 4ms/step - loss: 2.0472 - accuracy: 0.2674


[2.047179937362671, 0.26743361353874207]

# SUMMARY

Построено 3 разных сети с сопостовимым количеством параметров, на одних и тех же наборах данных, произведена их оценка по метрике accuracy.

Accuracy:

1. RNN (LSTM): 0.7384955883026123
2. CNN       : 0.5718
3. MLP       : 0.26743361353874207

В соответсвие с оценками можно считать решение основаное на рекурентных нейронных сетях наиболее перспективным, однако не стоит забывать, что набор данных был не сбалансирован и изменение соотношений данных может снизить точность предсказаний.


Спасибо за интересную задачу! Буду рад продолжить сотрудничество с вами!)

